In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# --- paramètres
range_theta = 720   # 0 à 720° (2 tours)
step_deg    = 5     # pas pour le slider

# points pour le cercle
theta_circle = np.linspace(0, 2*np.pi, 361)
x_circle = np.cos(theta_circle)
y_circle = np.sin(theta_circle)

# points pour la courbe sinus complète
theta_full_deg = np.linspace(0, range_theta, 800)
theta_full_rad = np.deg2rad(theta_full_deg)
sin_full = np.sin(theta_full_rad)

# valeurs d'angle pour les frames
theta_deg_vals = np.arange(0, range_theta + 1, step_deg)

# --- figure avec deux sous-graphiques : cercle | sinus
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "xy"}, {"type": "xy"}]],
    column_widths=[0.5, 0.5],
    subplot_titles=("Cercle trigonométrique", "Sinus")
)

# cercle fixe
fig.add_trace(
    go.Scatter(x=x_circle, y=y_circle, mode="lines", name="Cercle"),
    row=1, col=1
)

# courbe de sinus fixe
fig.add_trace(
    go.Scatter(x=theta_full_deg, y=sin_full, mode="lines", name="sin(θ)"),
    row=1, col=2
)

# --- éléments mobiles initiaux (θ = 60°)
theta0_deg = 60
theta0_rad = np.deg2rad(theta0_deg)
x0 = np.cos(theta0_rad)
y0 = np.sin(theta0_rad)

# rayon sur le cercle
rayon = go.Scatter(
    x=[0, x0], y=[0, y0],
    mode="lines",
    line=dict(width=3, color="red"),
    showlegend=False,
    name="rayon"
)
# point sur le cercle
point_cercle = go.Scatter(
    x=[x0], y=[y0],
    mode="markers",
    marker=dict(size=8, color="red"),
    name="point cercle"
)
# projection verticale (sinus)
proj_sin = go.Scatter(
    x=[0, 0], y=[0, y0],
    mode="lines",
    line=dict(width=3, color="orange"),
    name="sin θ",
    showlegend=False
)
# point sur la courbe sinus
point_sin = go.Scatter(
    x=[theta0_deg], y=[np.sin(theta0_rad)],
    mode="markers",
    marker=dict(size=8, color="red"),
    showlegend=False,
    name="point sin"
)

fig.add_trace(rayon,        row=1, col=1)
fig.add_trace(point_cercle, row=1, col=1)
fig.add_trace(proj_sin,     row=1, col=1)
fig.add_trace(point_sin,    row=1, col=2)

# --- frames pour chaque valeur de θ
frames = []
for deg in theta_deg_vals:
    t = np.deg2rad(deg)
    x = np.cos(t)
    y = np.sin(t)

    frames.append(
        go.Frame(
            data=[
                go.Scatter(x=x_circle, y=y_circle),
                go.Scatter(x=theta_full_deg, y=sin_full),
                go.Scatter(x=[0, x], y=[0, y]),
                go.Scatter(x=[x], y=[y]),
                go.Scatter(x=[0, 0], y=[0, y]),
                go.Scatter(x=[deg], y=[np.sin(t)]),
            ],
            name=str(deg)
        )
    )

fig.frames = frames

# --- slider + boutons Play / Pause
slider_steps = []
for deg in theta_deg_vals:
    step = {
        "method": "animate",
        "args": [[str(deg)],
                 {"mode": "immediate",
                  "frame": {"duration": 0, "redraw": True},
                  "transition": {"duration": 0}}],
        "label": str(deg)
    }
    slider_steps.append(step)

sliders = [{
    "active": list(theta_deg_vals).index(theta0_deg) if theta0_deg in theta_deg_vals else 0,
    "currentvalue": {"prefix": "θ = "},
    "pad": {"t": 40},
    "steps": slider_steps,
    "x": 0.1,
    "y": -0.1,
    "len": 0.8,
}]

fig.update_layout(
    sliders=sliders,
    updatemenus=[{
        "type": "buttons",
        "direction": "left",
        "x": 0.5,
        "y": 1.15,
        "showactive": False,
        "buttons": [
            {
                "label": "▶ Play",
                "method": "animate",
                "args": [None,
                         {"fromcurrent": True,
                          "frame": {"duration": 50, "redraw": True},
                          "transition": {"duration": 0}}]
            },
            {
                "label": "⏸ Pause",
                "method": "animate",
                "args": [[None],
                         {"frame": {"duration": 0, "redraw": False},
                          "mode": "immediate"}]
            }
        ]
    }],
    xaxis=dict(scaleanchor="y", range=[-1.2, 1.2]),
    yaxis=dict(range=[-1.2, 1.2]),
    xaxis2=dict(range=[0, range_theta]),
    yaxis2=dict(range=[-1.1, 1.1]),
    title="Trigonométrie interactive (Plotly + HTML)"
)

fig.show()

# Export HTML autonome
fig.write_html("trigo_interactif.html", include_plotlyjs="cdn")
print("Fichier HTML généré : trigo_interactif.html")
